# Embeddings

## Word2Vec

Векторные модели, которые мы рассматривали до этого (tf-idf, BOW), условно называются *счётными*. Они основываются на том, что так или иначе "считают" слова и их соседей, и на основе этого строят вектора для слов. 

Другой класс моделей, который более повсевмёстно распространён на сегодняшний день, называется *предсказательными* (или *нейронными*) моделями. Идея этих моделей заключается в использовании нейросетевых архитектур, которые "предсказывают" (а не считают) соседей слов. Одной из самых известных таких моделей является word2vec. Технология основана на нейронной сети, предсказывающей вероятность встретить слово в заданном контексте. Этот инструмент был разработан группой исследователей Google в 2013 году, руководителем проекта был Томаш Миколов (сейчас работает в Facebook). Вот две самые главные статьи:

* [Efficient Estimation of Word Representations in Vector Space](https://arxiv.org/pdf/1301.3781.pdf)
* [Distributed Representations of Words and Phrases and their Compositionality](https://arxiv.org/abs/1310.4546)

$$\hat{P}(w_1^T) = \prod_{t=1}^T \hat{P}(w_t \mid w_1^{t-1})$$

Полученные таким образом вектора называются *распределенными представлениями слов*, или **эмбеддингами**.


### Как это обучается?
Мы задаём вектор для каждого слова с помощью матрицы $w$ и вектор контекста с помощью матрицы $W$. По сути, word2vec является обобщающим названием для двух архитектур Skip-Gram и Continuous Bag-Of-Words (CBOW).  

**CBOW** предсказывает текущее слово, исходя из окружающего его контекста. 

**Skip-gram**, наоборот, использует текущее слово, чтобы предугадывать окружающие его слова. 

### Как это работает?
Word2vec принимает большой текстовый корпус в качестве входных данных и сопоставляет каждому слову вектор, выдавая координаты слов на выходе. Сначала он создает словарь, «обучаясь» на входных текстовых данных, а затем вычисляет векторное представление слов. Векторное представление основывается на контекстной близости: слова, встречающиеся в тексте рядом с одинаковыми словами (а следовательно, согласно дистрибутивной гипотезе, имеющие схожий смысл), в векторном представлении будут иметь близкие координаты векторов-слов. Для вычисления близости слов используется косинусное расстояние между их векторами.


С помощью дистрибутивных векторных моделей можно строить семантические пропорции (они же аналогии) и решать примеры:

* *король: мужчина = королева: женщина* 
 $\Rightarrow$ 
* *король - мужчина + женщина = королева*

![w2v](https://cdn-images-1.medium.com/max/2600/1*sXNXYfAqfLUeiDXPCo130w.png)

### Проблемы
Невозможно установить тип семантических отношений между словами: синонимы, антонимы и т.д. будут одинаково близки, потому что обычно употребляются в схожих контекстахю Поэтому близкие в векторном пространстве слова называют *семантическими ассоциатами*. Это значит, что они семантически связаны, но как именно — непонятно.


## RusVectōrēs


На сайте [RusVectōrēs](https://rusvectores.org/ru/) собраны предобученные на различных данных модели для русского языка, а также можно поискать наиболее близкие слова к заданному, посчитать семантическую близость нескольких слов и порешать примеры с помощью «калькулятором семантической близости».


Для других языков также можно найти предобученные модели — например, модели [fastText](https://fasttext.cc/docs/en/english-vectors.html) и [GloVe](https://nlp.stanford.edu/projects/glove/) (о них чуть дальше).

## Gensim

Использовать предобученную модель эмбеддингов или обучить свою можно с помощью библиотеки `gensim`. Вот [ее документация](https://radimrehurek.com/gensim/models/word2vec.html).

### Как использовать готовую модель

Модели word2vec бывают разных форматов:

* .vec.gz — обычный файл
* .bin.gz — бинарник

Загружаются они с помощью одного и того же гласса `KeyedVectors`, меняется только параметр `binary` у функции `load_word2vec_format`. 

Если же эмбеддинги обучены **не** с помощью word2vec, то для загрузки нужно использовать функцию `load`. Т.е. для загрузки предобученных эмбеддингов *glove, fasttext, bpe* и любых других нужна именно она.

Скачаем с RusVectōrēs модель для русского языка, обученную на НКРЯ образца 2015 г. 

In [1]:
import re
import gensim
import logging
import nltk.data 
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from gensim.models import word2vec
from nltk.tokenize import sent_tokenize, RegexpTokenizer

In [54]:
urllib.request.urlretrieve("http://rusvectores.org/static/models/rusvectores2/ruscorpora_mystem_cbow_300_2_2015.bin.gz", "ruscorpora_mystem_cbow_300_2_2015.bin.gz")

('ruscorpora_mystem_cbow_300_2_2015.bin.gz',
 <http.client.HTTPMessage at 0x138433e10>)

In [55]:
model_path = 'ruscorpora_mystem_cbow_300_2_2015.bin.gz'

model_ru = gensim.models.KeyedVectors.load_word2vec_format(model_path, binary=True)

In [56]:
words = ['день_S', 'ночь_S', 'человек_S', 'семантика_S', 'биткоин_S']

Частеречные тэги нужны, поскольку это специфика скачанной модели - она была натренирована на словах, аннотированных их частями речи (и лемматизированных). **NB!** В названиях моделей на `rusvectores` указано, какой тегсет они используют (mystem, upos и т.д.)

Попросим у модели 10 ближайших соседей для каждого слова и коэффициент косинусной близости для каждого:

In [57]:
for word in words:
    # есть ли слово в модели? 
    if word in model_ru:
        print(word)
        # смотрим на вектор слова (его размерность 300, смотрим на первые 10 чисел)
        print(model_ru[word][:10])
        # выдаем 10 ближайших соседей слова:
        for i in model_ru.most_similar(positive=[word], topn=10):
            # слово + коэффициент косинусной близости
            print(i[0], i[1])
        print('\n')
    else:
        # Увы!
        print('Увы, слова "%s" нет в модели!' % word)

день_S
[-0.02580778  0.00970898  0.01941961 -0.02332282  0.02017624  0.07275085
 -0.01444375  0.03316632  0.01242602  0.02833412]
неделя_S 0.7165195941925049
месяц_S 0.631048858165741
вечер_S 0.5828738808631897
утро_S 0.5676206946372986
час_S 0.5605547428131104
минута_S 0.5297019481658936
гекатомбеон_S 0.4897990822792053
денек_S 0.48224714398384094
полчаса_S 0.48217129707336426
ночь_S 0.478074848651886


ночь_S
[-0.00688948  0.00408364  0.06975466 -0.00959525  0.0194835   0.04057068
 -0.00994112  0.06064967 -0.00522624  0.00520327]
вечер_S 0.6946248412132263
утро_S 0.57301926612854
ноченька_S 0.5582467317581177
рассвет_S 0.5553584098815918
ночка_S 0.5351512432098389
полдень_S 0.5334426760673523
полночь_S 0.4786943793296814
день_S 0.4780748784542084
сумерки_S 0.4390218257904053
фундерфун_S 0.4340824782848358


человек_S
[ 0.02013756 -0.02670703 -0.02039861 -0.05477146  0.00086402 -0.01636335
  0.04240306 -0.00025525 -0.14045681  0.04785006]
женщина_S 0.5979774594306946
парень_S 0.499178

Находим косинусную близость пары слов:

In [6]:
print(model_ru.similarity('человек_S', 'обезьяна_S'))

0.2389560934322063


Что получится, если вычесть из пиццы Италию и прибавить Сибирь?

* positive — вектора, которые мы складываем
* negative — вектора, которые вычитаем

In [7]:
print(model_ru.most_similar(positive=['пицца_S', 'сибирь_S'], negative=['италия_S'])[0][0])

пельмень_S


In [8]:
model_ru.doesnt_match('пицца_S пельмень_S хот-дог_S ананас_S'.split())

'ананас_S'

### Как обучить свою модель

В качестве обучающих данных возьмем размеченные и неразмеченные отзывы о фильмах (датасет взят с Kaggle).

In [14]:
! wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/unlabeledTrainData.tsv

--2019-09-08 12:47:09--  https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/unlabeledTrainData.tsv
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 67281491 (64M) [text/plain]
Сохранение в: «unlabeledTrainData.tsv.1»

unlabeledTrainData. 100%[===================>]  64,16M  4,39MB/s    за 17s     

2019-09-08 12:47:29 (3,72 MB/s) - «unlabeledTrainData.tsv.1» сохранён [67281491/67281491]



In [22]:
data = pd.read_csv("unlabeledTrainData.tsv", header=0, delimiter="\t", quoting=3)

len(data)

50000

In [18]:
data.head()

,id,review
0,"""9999_0""","""Watching Time Chasers, it obvious that it was..."
1,"""45057_0""","""I saw this film about 20 years ago and rememb..."
2,"""15561_0""","""Minor Spoilers<br /><br />In New York, Joan B..."
3,"""7161_0""","""I went to see this film with a great deal of ..."
4,"""43971_0""","""Yes, I agree with everyone on this site this ..."


Убираем из данных ссылки, html-разметку и небуквенные символы, а затем приводим все к нижнему регистру и токенизируем. На выходе получается массив из предложений, каждое из которых представляет собой массив слов. Здесь используется токенизатор из библиотеки `nltk`. 

In [19]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

def review_to_wordlist(review, remove_stopwords=False ):
    # убираем ссылки вне тегов
    review = re.sub(r"http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+", " ", review)
    review_text = BeautifulSoup(review, "lxml").get_text()
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    words = review_text.lower().split()
    if remove_stopwords:
        stops = stopwords.words("english")
        words = [w for w in words if not w in stops]
    return(words)

def review_to_sentences(review, tokenizer, remove_stopwords=False):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence) > 0:
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    return sentences

In [20]:
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

sentences = []  

print("Parsing sentences from training set...")
for review in data["review"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set...


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'..'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)


In [21]:
print(len(sentences))
print(sentences[0])

528987
['watching', 'time', 'chasers', 'it', 'obvious', 'that', 'it', 'was', 'made', 'by', 'a', 'bunch', 'of', 'friends']


In [23]:
# это понадобится нам позже

with open('clean_text.txt', 'w') as f:
    for s in sentences[:5000]:
        f.write(' '.join(s))
        f.write('\n')

Обучаем и сохраняем модель. 


Основные параметры:
* данные должны быть итерируемым объектом 
* size — размер вектора, 
* window — размер окна наблюдения,
* min_count — мин. частотность слова в корпусе,
* sg — используемый алгоритм обучения (0 — CBOW, 1 — Skip-gram),
* sample — порог для downsampling'a высокочастотных слов,
* workers — количество потоков,
* alpha — learning rate,
* iter — количество итераций,
* max_vocab_size — позволяет выставить ограничение по памяти при создании словаря (т.е. если ограничение привышается, то низкочастотные слова будут выбрасываться). Для сравнения: 10 млн слов = 1Гб RAM.

**NB!** Обратите внимание, что тренировка модели не включает препроцессинг! Это значит, что избавляться от пунктуации, приводить слова к нижнему регистру, лемматизировать их, проставлять частеречные теги придется до тренировки модели (если, конечно, это необходимо для вашей задачи). Т.е. в каком виде слова будут в исходном тексте, в таком они будут и в модели.

In [24]:
print("Training model...")

%time model_en = word2vec.Word2Vec(sentences, workers=4, size=300, min_count=10, window=10, sample=1e-3)

Training model...
CPU times: user 2min 52s, sys: 1.19 s, total: 2min 54s
Wall time: 58.7 s


Смотрим, сколько в модели слов.

In [25]:
print(len(model_en.wv.vocab))

28308


Попробуем оценить модель вручную, порешав примеры. Несколько дано ниже, попробуйте придумать свои.

In [26]:
print(model_en.wv.most_similar(positive=["woman", "actor"], negative=["man"], topn=1))
print(model_en.wv.most_similar(positive=["dogs", "man"], negative=["dog"], topn=1))

print(model_en.wv.most_similar("usa", topn=3))

print(model_en.wv.doesnt_match("comedy thriller western novel".split()))

[('actress', 0.7690370082855225)]
[('men', 0.6405411958694458)]
[('germany', 0.7654047012329102), ('europe', 0.7413791418075562), ('canada', 0.7300304174423218)]
novel


### Как дообучить существующую модель

При тренировке модели "с нуля" веса инициализируются случайно, однако можно использовать для инициализации векторов веса из предобученной модели, таким образом как бы дообучая ее.

Сначала посмотрим близость какой-нибудь пары слов в имеющейс модели, чтобы потом сравнить результат с дообученной.

In [48]:
model_en.wv.similarity('lion', 'rabbit')

0.2693721120152248

В качестве дополнительных данных для обучения возьмем английский текст «Алисы в Зазеркалье».

In [32]:
! wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/alice.txt

--2019-09-08 12:55:27--  https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/train/alice.txt
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 151.101.0.133, 151.101.128.133, 151.101.192.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 167631 (164K) [text/plain]
Сохранение в: «alice.txt.1»

alice.txt.1         100%[===================>] 163,70K  --.-KB/s    за 0,09s   

2019-09-08 12:55:27 (1,83 MB/s) - «alice.txt.1» сохранён [167631/167631]



In [35]:
with open("alice.txt", 'r', encoding='utf-8') as f:
    text = f.read()

text = re.sub('\n', ' ', text)
sents = sent_tokenize(text)

punct = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~„“«»†*—/\-‘’'
clean_sents = []

for sent in sents:
    s = [w.lower().strip(punct) for w in sent.split()]
    clean_sents.append(s)
    
print(clean_sents[:2])

[['through', 'the', 'looking-glass', 'by', 'lewis', 'carroll', 'chapter', 'i', 'looking-glass', 'house', 'one', 'thing', 'was', 'certain', 'that', 'the', 'white', 'kitten', 'had', 'had', 'nothing', 'to', 'do', 'with', 'it', '', 'it', 'was', 'the', 'black', 'kitten’s', 'fault', 'entirely'], ['for', 'the', 'white', 'kitten', 'had', 'been', 'having', 'its', 'face', 'washed', 'by', 'the', 'old', 'cat', 'for', 'the', 'last', 'quarter', 'of', 'an', 'hour', 'and', 'bearing', 'it', 'pretty', 'well', 'considering', 'so', 'you', 'see', 'that', 'it', 'couldn’t', 'have', 'had', 'any', 'hand', 'in', 'the', 'mischief']]


Чтобы дообучить модель, надо сначала ее сохранить, а потом загрузить. Все параметры тренировки (размер вектора, мин. частота слова и т.п.) будут взяты из загруженной модели, т.е. задать их заново нельзя.

**NB!** Дообучить можно только полную модель, а `KeyedVectors` — нельзя. Поэтому сохранять модель нужно в соотвествующем формате. Подробнее о разнице [вот тут](https://radimrehurek.com/gensim/models/keyedvectors.html).

In [36]:
model_path = "movie_reviews.model"

print("Saving model...")
model_en.save(model_path)

Saving model...


In [37]:
model = word2vec.Word2Vec.load(model_path)

model.build_vocab(clean_sents, update=True)
model.train(clean_sents, total_examples=model.corpus_count, epochs=5)

(96931, 150225)

Лев и кролик стали ближе друг к другу!

In [49]:
model.wv.similarity('lion', 'rabbit')

0.28044813294218507

Можно нормализовать вектора, тогда модель будет занимать меньше RAM. Однако после этого её нельзя дотренировывать. Здесь используется L2-нормализация: вектора нормализуются так, что если сложить квадраты всех элементов вектора, в сумме получится 1. 

Кроме того, сохраним не полные вектора, а `KeyedVectors`.

In [50]:
model.init_sims(replace=True)
model_path = "movies_alice.bin"

print("Saving model...")
model_en.wv.save_word2vec_format(model_path, binary=True)

Saving model...


## Оценка

Это, конечно, хорошо, но как понять, какая модель лучше? Или вот, например, я сделал свою модель, а как понять, насколько она хорошая?

Для этого существуют специальные датасеты для оценки качества дистрибутивных моделей. Основных два: один измеряет точность решения задач на аналогии (про Россию и пельмени), а второй используется для оценки коэффициента семантической близости. 

### Word Similarity

Этот метод заключается в том, чтобы оценить, насколько представления о семантической близости слов в модели соотносятся с "представлениями" людей.

| слово 1    | слово 2    | близость | 
|------------|------------|----------|
| кошка      | собака     | 0.7      |  
| чашка      | кружка     | 0.9      |       

Для каждой пары слов из заранее заданного датасета мы можем посчитать косинусное расстояние, и получить список таких значений близости. При этом у нас уже есть список значений близостей, сделанный людьми. Мы можем сравнить эти два списка и понять, насколько они похожи (например, посчитав корреляцию). Эта мера схожести должна говорить о том, насколько модель хорошо моделирует расстояния о слова.

### Аналогии

Другая популярная задача для "внутренней" оценки называется задачей поиска аналогий. Как мы уже разбирали выше, с помощью простых арифметических операций мы можем модифицировать значение слова. Если заранее собрать набор слов-модификаторов, а также слов, которые мы хотим получить в результаты модификации, то на основе подсчёта количества "попаданий" в желаемое слово мы можем оценить, насколько хорошо работает модель.

В качестве слов-модификатор мы можем использовать семантические аналогии. Скажем, если у нас есть некоторое отношение "страна-столица", то для оценки модели мы можем использовать пары наподобие "Россия-Москва", "Норвегия-Осло", и т.д. Датасет будет выглядеть следующм образом:

| слово 1    | слово 2    | отношение     | 
|------------|------------|---------------|
| Россия     | Москва     | страна-столица|  
| Норвегия   | Осло       | страна-столица|

Рассматривая случайные две пары из этого набора, мы хотим, имея триплет (Россия, Москва, Норвегия) хотим получить слово "Осло", т.е. найти такое слово, которое будет находиться в том же отношении со словом "Норвегия", как "Россия" находится с Москвой. 

Датасеты для русского языка можно скачать на странице с моделями на RusVectores. Посчитаем качество нашей модели НКРЯ на датасете про аналогии:

In [51]:
! wget https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/evaluation/ru_analogy_tagged.txt

--2019-09-08 12:58:41--  https://raw.githubusercontent.com/ancatmara/data-science-nlp/master/data/w2v/evaluation/ru_analogy_tagged.txt
Распознаётся raw.githubusercontent.com (raw.githubusercontent.com)… 151.101.0.133, 151.101.64.133, 151.101.192.133, ...
Подключение к raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 871776 (851K) [text/plain]
Сохранение в: «ru_analogy_tagged.txt»

ru_analogy_tagged.t 100%[===================>] 851,34K  5,43MB/s    за 0,2s    

2019-09-08 12:58:42 (5,43 MB/s) - «ru_analogy_tagged.txt» сохранён [871776/871776]



In [58]:
res = model_ru.accuracy('ru_analogy_tagged.txt')

In [23]:
print(res[4]['incorrect'][:10])

[('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ДЕД_S', 'БАБКА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'КОРОЛЬ_S', 'КОРОЛЕВА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ПРИНЦ_S', 'ПРИНЦЕССА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ОТЧИМ_S', 'МАЧЕХА_S'), ('МАЛЬЧИК_S', 'ДЕВОЧКА_S', 'ПАСЫНОК_S', 'ПАДЧЕРИЦА_S'), ('БРАТ_S', 'СЕСТРА_S', 'ДЕД_S', 'БАБКА_S'), ('БРАТ_S', 'СЕСТРА_S', 'ОТЧИМ_S', 'МАЧЕХА_S'), ('БРАТ_S', 'СЕСТРА_S', 'ПАСЫНОК_S', 'ПАДЧЕРИЦА_S'), ('ПАПА_S', 'МАМА_S', 'ДЕД_S', 'БАБКА_S'), ('ПАПА_S', 'МАМА_S', 'ОТЧИМ_S', 'МАЧЕХА_S')]


## Визуализация

На полученную модель можно посмотреть, визуализировав ее, например, на плоскости.
### t-SNE

**t-SNE**  (*t-distributed Stochastic Neighbor Embedding*) — техника нелинейного снижения размерности и визуализации многомерных переменных. Она разработана специально для данных высокой размерности Л. ван дер Маатеном и Д. Хинтоном, [вот их статья](http://jmlr.org/papers/volume9/vandermaaten08a/vandermaaten08a.pdf). t-SNE — это итеративный алгоритм, основанный на вычислении попарных расстояний между всеми объектами (в том числе поэтому он довольно медленный).


Изобразим на плоскости 1000 самых частотных слов из коллекции текстов про кино:

In [4]:
from nltk import FreqDist
from tqdm import tqdm_notebook as tqdm
from sklearn.manifold import TSNE

top_words = []


fd = FreqDist()
for s in tqdm(sentences):
    fd.update(s)

for w in fd.most_common(1000):
    top_words.append(w[0])
    
print(top_words[:50:])
top_words_vec = model[top_words]

In [36]:
%%time
tsne = TSNE(n_components=2, random_state=0)
top_words_tsne = tsne.fit_transform(top_words_vec)

CPU times: user 20.8 s, sys: 1.1 s, total: 21.9 s
Wall time: 22.2 s


In [39]:
from bokeh.models import ColumnDataSource, LabelSet
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (eng model, top1000 words)")

source = ColumnDataSource(data=dict(x1=top_words_tsne[:,0],
                                    x2=top_words_tsne[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

Чтобы вычислить преобразование t-SNE быстрее (и иногда еще и эффективнее), можно сперва снизить размерность исходных данных с помощью, например, SVD, и потом применять t-SNE:

In [42]:
from sklearn.decomposition import TruncatedSVD

svd_50 = TruncatedSVD(n_components=50)
top_words_vec_50 = svd_50.fit_transform(top_words_vec)
top_words_tsne2 = TSNE(n_components=2, random_state=0).fit_transform(top_words_vec_50)

In [43]:
output_notebook()

p = figure(tools="pan,wheel_zoom,reset,save",
           toolbar_location="above",
           title="word2vec T-SNE (eng model, top1000 words, +SVD)")

source = ColumnDataSource(data=dict(x1=top_words_tsne2[:,0],
                                    x2=top_words_tsne2[:,1],
                                    names=top_words))

p.scatter(x="x1", y="x2", size=8, source=source)

labels = LabelSet(x="x1", y="x2", text="names", y_offset=6,
                  text_font_size="8pt", text_color="#555555",
                  source=source, text_align='center')
p.add_layout(labels)

show(p)

Loading BokehJS ...

## FastText

FastText использует не только эмбеддинги слов, но и эмбеддинги n-грам. В корпусе каждое слово автоматически представляется в виде набора символьных n-грамм. Скажем, если мы установим n=3, то вектор для слова "where" будет представлен суммой векторов следующих триграм: "<wh", "whe", "her", "ere", "re>" (где "<" и ">" символы, обозначающие начало и конец слова). Благодаря этому мы можем также получать вектора для слов, отсутствуюших в словаре, а также эффективно работать с текстами, содержащими ошибки и опечатки.

* [Статья](https://aclweb.org/anthology/Q17-1010)
* [Сайт](https://fasttext.cc/)
* [Тьюториал](https://fasttext.cc/docs/en/support.html)
* [Вектора для 157 языков](https://fasttext.cc/docs/en/crawl-vectors.html)
* [Вектора, обученные на википедии](https://fasttext.cc/docs/en/pretrained-vectors.html) (отдельно для 294 разных языков)
* [Репозиторий](https://github.com/facebookresearch/fasttext)

Есть библиотека `fasttext` для питона (с готовыми моделями можно работать и через `gensim`).

In [59]:
import fasttext

# так можно обучить свою модель 
ft_model = fasttext.train_unsupervised('clean_text.txt', minn=3, maxn=4, dim=300)

In [60]:
ft_model.get_word_vector("movie")

array([-9.26938560e-03,  1.49881303e-01, -1.68044731e-01, -3.42621356e-02,
        3.12056728e-02, -2.70286333e-02, -2.25703284e-01, -4.61917184e-02,
       -4.98749875e-02,  9.88036618e-02,  9.06925574e-02,  8.58290568e-02,
        2.65159886e-02,  5.39430939e-02,  1.10680394e-01, -8.62991624e-03,
        8.84623155e-02, -5.26909232e-02,  1.35063052e-01,  9.35826823e-02,
        8.97094384e-02, -3.88720594e-02,  1.27567768e-01, -8.02966207e-03,
        1.01278596e-01,  5.19590750e-02,  1.49484232e-01,  6.19858364e-03,
       -1.42766461e-01,  7.11348653e-03,  8.17533061e-02, -1.15331080e-04,
        5.02409115e-02, -2.74927262e-02, -1.13569256e-02, -7.69021511e-02,
        1.05734564e-01, -5.88382855e-02, -9.06331465e-02, -1.64451167e-01,
        1.75815925e-01,  8.27199519e-02, -2.97587663e-02,  1.57219544e-02,
        5.97899556e-02, -3.27031128e-02, -2.07461156e-02,  5.91145344e-02,
       -6.80698380e-02, -9.68456417e-02,  1.12470143e-01, -1.10266678e-01,
        1.70350075e-02,  

In [61]:
ft_model.get_nearest_neighbors('actor')

[(0.999930202960968, 'director'),
 (0.9998905658721924, 'actresses'),
 (0.9998632669448853, 'perfectly'),
 (0.9998623728752136, 'perfect'),
 (0.9998612999916077, 'predator'),
 (0.99985271692276, 'actual'),
 (0.9998513460159302, 'directors'),
 (0.9998440742492676, 'superior'),
 (0.9998432397842407, 'dimensional'),
 (0.9998407959938049, 'doctor')]

In [62]:
ft_model.get_analogies("woman", "man", "actor")

[(0.9997962713241577, 'act'),
 (0.9997937679290771, 'written'),
 (0.9997843503952026, 'write'),
 (0.9997797012329102, 'merely'),
 (0.9997739791870117, 'barely'),
 (0.9997737407684326, 'actors'),
 (0.9997718334197998, 'utterly'),
 (0.9997681379318237, 'admittedly'),
 (0.999761700630188, 'deliberately'),
 (0.9997599720954895, 'ugly')]

In [63]:
# проблема с опечатками решена

ft_model.get_nearest_neighbors('actr')

[(0.999885082244873, 'act'),
 (0.999877393245697, 'actors'),
 (0.9998356103897095, 'actual'),
 (0.9998084306716919, 'actress'),
 (0.9998043179512024, 'directors'),
 (0.9997732639312744, 'perspective'),
 (0.9997712969779968, 'direct'),
 (0.9997701048851013, 'incredibly'),
 (0.9997696876525879, 'inevitable'),
 (0.9997684359550476, 'attractive')]

In [64]:
# проблема с out of vocabulary словами - тоже

ft_model.get_nearest_neighbors('moviegeek')

[(0.9999075531959534, 'reviews'),
 (0.9998737573623657, 'review'),
 (0.9998543858528137, 'remake'),
 (0.9998310804367065, 'watchable'),
 (0.9998247623443604, 'how'),
 (0.9998225569725037, 'now'),
 (0.999821126461029, 'like'),
 (0.9998193383216858, 'movie'),
 (0.9998154044151306, 'waste'),
 (0.9998078942298889, 'said')]

In [3]:
!wget -O positive.csv https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0

--2019-09-07 17:04:20--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv?dl=0
Распознаётся www.dropbox.com (www.dropbox.com)… 162.125.70.1
Подключение к www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 301 Moved Permanently
Адрес: /s/raw/fnpq3z4bcnoktiv/positive.csv [переход]
--2019-09-07 17:04:21--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Повторное использование соединения с www.dropbox.com:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://uc6e0483c7461e65c1c3d732aa19.dl.dropboxusercontent.com/cd/0/inline/AoF8AacHXiZa2jyu4tqZjqBWClQ9i0qCeIW0VkLfMSuZLPrZT5HAR2D1aoILOlkKM9FIBlfePxThsc5y9hrkngjbm5TKqK4vvVPVX7fsQ-bEAA/file# [переход]
--2019-09-07 17:04:21--  https://uc6e0483c7461e65c1c3d732aa19.dl.dropboxusercontent.com/cd/0/inline/AoF8AacHXiZa2jyu4tqZjqBWClQ9i0qCeIW0VkLfMSuZLPrZT5HAR2D1aoILOlkKM9FIBlfePxThsc5y9hrkngjbm5TKqK4vvVPVX7fsQ-bEAA/file
Распознаётся uc6e0483c7461e65

In [5]:
!wget -O negative.csv https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0

--2019-09-07 17:04:48--  https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv?dl=0
Распознаётся www.dropbox.com (www.dropbox.com)… 162.125.70.1
Подключение к www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 301 Moved Permanently
Адрес: /s/raw/r6u59ljhhjdg6j0/negative.csv [переход]
--2019-09-07 17:04:49--  https://www.dropbox.com/s/raw/r6u59ljhhjdg6j0/negative.csv
Повторное использование соединения с www.dropbox.com:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://ucac5144b5879d47d6296c4bef32.dl.dropboxusercontent.com/cd/0/inline/AoGBe7-mm6lRHQ8AnaSbbNYilOBaAo0Yk3JBIv2SuJEfWAEy1NPN2lkGbwyX-a87SD06TXeFVbqDfN7grHh9kB5ObxNxHPWJxm3W37uatPatGA/file# [переход]
--2019-09-07 17:04:50--  https://ucac5144b5879d47d6296c4bef32.dl.dropboxusercontent.com/cd/0/inline/AoGBe7-mm6lRHQ8AnaSbbNYilOBaAo0Yk3JBIv2SuJEfWAEy1NPN2lkGbwyX-a87SD06TXeFVbqDfN7grHh9kB5ObxNxHPWJxm3W37uatPatGA/file
Распознаётся ucac5144b5879d47

In [4]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = positive.append(negative)
df.head()

,text,label
0,"@first_timee хоть я и школота, но поверь, у на...",positive
1,"Да, все-таки он немного похож на него. Но мой ...",positive
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive


Проведем стандартный препроцессинг:

In [5]:
from pymystem3 import Mystem
from functools import lru_cache
from multiprocessing import Pool
import numpy as np
from sklearn.model_selection import train_test_split

m = Mystem()

regex = re.compile("[А-Яа-я:=!\)\()A-z\_\%/|]+")

def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text))
    except:
        return ""

In [6]:
@lru_cache(maxsize=128)
def lemmatize(text, mystem=m):
    try:
        return " ".join(m.lemmatize(text)).strip()  
    except:
        return " "    

In [7]:
def clean_tetx(text):
    return lemmatize(words_only(text))

In [10]:
with Pool(8) as p:
    lemmas = list(tqdm(p.imap(clean_tetx, df['text']), total=len(df)))
    
df['lemmas'] = lemmas
df.head()

HBox(children=(IntProgress(value=0, max=226834), HTML(value='')))

,text,label,lemmas
0,"@first_timee хоть я и школота, но поверь, у на...",positive,first _ timee хоть я и школоть но ...
1,"Да, все-таки он немного похож на него. Но мой ...",positive,да весь таки он немного похожий на...
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,positive,RT KatiaCheh : ну ты идиотка ) я исп...
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",positive,RT digger : кто то в угол сидеть ...
4,@irina_dyshkant Вот что значит страшилка :D\nН...,positive,irina _ dyshkant вот что значить страш...


Запишем полученные данные в формате для обучения классификатора:

In [11]:
X = df.lemmas.tolist()
y = df.label.tolist()

X, y = np.array(X), np.array(y)

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33)
print ("total train examples %s" % len(y_train))
print ("total test examples %s" % len(y_test))

total train examples 151978
total test examples 74856


In [12]:
with open('data.train.txt', 'w+') as outfile:
    for i in range(len(X_train)):
        outfile.write('__label__' + y_train[i] + ' '+ X_train[i] + '\n')
    

with open('test.txt', 'w+') as outfile:
    for i in range(len(X_test)):
        outfile.write('__label__' + y_test[i] + ' ' + X_test[i] + '\n')

In [18]:
classifier = fasttext.train_supervised('data.train.txt')
result = classifier.test('test.txt')

print('P@1:', result[1])
print('R@1:', result[2])
print('Number of examples:', result[0])

P@1: 0.9968739980763065
R@1: 0.9968739980763065
Number of examples: 74856


## Задание

1. Мы будем работать с (частичными) данными lenta.ru отсюда: https://www.kaggle.com/yutkin/corpus-of-russian-news-articles-from-lenta/
2. Проведите препроцессинг текста. Разбейте данные на train и test для задачи классификации (в качестве метки класса будем использовать поле topic). В качестве данных для классификации в пунктах 3 и 5 возьмите
    - только заголовки (title)
    - только тексты новости (text)
    - и то, и другое
3. Обучите fastText для классификации текстов по темам. Сравните качество для разных данных из п. 2.
4. Обучите свою модель w2v (или возьмите любую подходящую предобученную модель). Реализуйте функцию для вычисления вектора текста / заголовка / текста+заголовка как среднего вектора входящих в него слов. 
     - (Бонус) Модифицируйте функцию вычисления среднего вектора: взвешивайте вектора слов соответствующими весами tf-idf.
5. Обучите на полученных средних векторах алгоритм классификации, сравните полученное качество с классификатором fastText. 

In [67]:
! wget -O lenta-ru-news-part.csv https://www.dropbox.com/s/ja23c9l1ppo9ix7/lenta-ru-news-part.csv?dl=0

--2019-09-08 13:08:48--  https://www.dropbox.com/s/ja23c9l1ppo9ix7/lenta-ru-news-part.csv?dl=0
Распознаётся www.dropbox.com (www.dropbox.com)… 162.125.70.1
Подключение к www.dropbox.com (www.dropbox.com)|162.125.70.1|:443... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 301 Moved Permanently
Адрес: /s/raw/ja23c9l1ppo9ix7/lenta-ru-news-part.csv [переход]
--2019-09-08 13:08:48--  https://www.dropbox.com/s/raw/ja23c9l1ppo9ix7/lenta-ru-news-part.csv
Повторное использование соединения с www.dropbox.com:443.
HTTP-запрос отправлен. Ожидание ответа… 302 Found
Адрес: https://uc6df9979de2a2fa7a5a70f82cea.dl.dropboxusercontent.com/cd/0/inline/AoJl1P8bYh2VWwM0esHzv5iHTt_bC3pju0fDs5btgiYuRYKUQwOxbtCu1QVz64HGa0WowMLRlQbQob1y1066jNBtzqJWGjw2KnPAZ0pusjieLSVwtY8isfyIeuxEKia9x6E/file# [переход]
--2019-09-08 13:08:49--  https://uc6df9979de2a2fa7a5a70f82cea.dl.dropboxusercontent.com/cd/0/inline/AoJl1P8bYh2VWwM0esHzv5iHTt_bC3pju0fDs5btgiYuRYKUQwOxbtCu1QVz64HGa0WowMLRlQbQob1y1066jNBtzqJWGj

In [68]:
lenta = pd.read_csv('lenta-ru-news-part.csv', usecols=['title', 'text', 'topic'])
lenta.head()

,title,text,topic
0,Австрия не представила доказательств вины росс...,Австрийские правоохранительные органы не предс...,Спорт
1,Овечкин повторил свой рекорд,Капитан «Вашингтона» Александр Овечкин сделал...,Спорт
2,Названы регионы России с самым дорогим и дешев...,Производитель онлайн-касс «Эвотор» проанализир...,Экономика
3,Россию и Украину пригласили на переговоры по газу,Вице-президент Еврокомиссии Марош Шефчович при...,Экономика
4,Хоккеист НХЛ забросил шайбу с отрицательного угла,Нападающий клуба «Эдмонтон Ойлерс» Коннор Макд...,Спорт


In [69]:
len(lenta)

258297

In [70]:
lenta.topic.value_counts()

Экономика          79538
Спорт              64421
Культура           53803
Наука и техника    53136
Бизнес              7399
Name: topic, dtype: int64